In [2]:
import numpy as np
import matplotlib.pyplot as plt
import gudhi as gd
import glob
import sklearn.manifold as manifold

# Create a list of tuples, each containing:
# 1. A name identifying the sample size (e.g., '500')
# 2. The corresponding point cloud loaded from a NumPy file
samples = [
    (filename.split('_')[-1].split('.')[0], np.load(filename))
    for filename in glob.glob('dragon_vrip_sampled_*.npy')
]

# Sort samples by their numeric values (convert string names to integers for proper sorti
samples.sort(key=lambda x: int(x[0]))
print("Samples loaded and sorted")

l = len(samples)

# Use Multidimensional Scaling (MDS) to visualize topological differences between samples
# This will allow us to visualize the relationships between the samples based on their persistence intervals
# MDS (Multidimensional Scaling) reduces high-dimensional distance matrices to 2D for visualization
mds = manifold.MDS(
    n_components=2,        # Reduce to 2 dimensions for plotting
    max_iter=3000,         # Maximum iterations for convergence
    eps=1e-9,             # Convergence tolerance
    dissimilarity="precomputed",  # Use precomputed distance matrices B0, B1, B2
    n_jobs=1,             # Number of parallel jobs
    n_init=1              # Number of initializations (silences warnings)
)


Samples loaded and sorted


In [ ]:
# Create lists to store persistence intervals for each homological dimension
# pl0: connected components (dimension 0)
# pl1: loops/cycles (dimension 1) 
# pl2: voids/cavities (dimension 2)
pl0 = []
pl1 = []
pl2 = []

# Process each sample to extract persistence intervals
for name, points in samples:
    # Create Alpha Complex from point cloud
    ac = gd.AlphaComplex(points=points)
    
    # Build simplex tree for topological analysis
    st = ac.create_simplex_tree()
    
    # Compute persistent homology
    bc = st.persistence()
    
    # Extract persistence intervals for each dimension and store them
    pl0.append(st.persistence_intervals_in_dimension(0))  # Connected components
    pl1.append(st.persistence_intervals_in_dimension(1))  # Loops/cycles
    pl2.append(st.persistence_intervals_in_dimension(2))  # Voids/cavities
    
    print(f"Persistence intervals for {name} appended.")

# # Display the shape of persistence intervals for each sample and dimension
# for i, name in enumerate([s[0] for s in samples]):
#     print(f"{name}: dim0={pl0[i].shape}, dim1={pl1[i].shape}, dim2={pl2[i].shape}")

# Initialize distance matrices for each homological dimension
# B0: bottleneck distances for connected components (dimension 0)
# B1: bottleneck distances for loops/cycles (dimension 1) 
# B2: bottleneck distances for voids/cavities (dimension 2)
B0 = np.zeros((l, l))
B1 = np.zeros((l, l))
B2 = np.zeros((l, l))

# Compute pairwise bottleneck distances between persistence intervals
# Only compute upper triangle to avoid redundant calculations
for i in range(l):
    for j in range(i):
        # Calculate bottleneck distance between persistence intervals of samples i and j
        B0[i,j] = gd.bottleneck_distance(pl0[i], pl0[j])  # Dimension 0
        B1[i,j] = gd.bottleneck_distance(pl1[i], pl1[j])  # Dimension 1
        B2[i,j] = gd.bottleneck_distance(pl2[i], pl2[j])  # Dimension 2
        print(f"Bottleneck distances for {samples[i][0]} and {samples[j][0]} computed.")

# Make distance matrices symmetric by adding the transpose
# This fills in the lower triangle with the same values as the upper triangle
B0 = B0 + B0.transpose()
B1 = B1 + B1.transpose()
B2 = B2 + B2.transpose()

# Apply MDS to each bottleneck distance matrix to get 2D coordinates
X0 = mds.fit_transform(B0)  # 2D embedding for dimension 0 (connected components)
X1 = mds.fit_transform(B1)  # 2D embedding for dimension 1 (loops/cycles)
X2 = mds.fit_transform(B2)  # 2D embedding for dimension 2 (voids/cavities)

# Create a single figure with 3 vertically stacked subplots
fig, axes = plt.subplots(3, 1, figsize=(10, 15))
sample_names = [s[0] for s in samples]  # Extract sample names for labeling

# Plot dimension 0 embedding (connected components)
axes[0].scatter(X0[:, 0], X0[:, 1], c=range(l), cmap='viridis')
for i, name in enumerate(sample_names):
    # Add sample name labels with slight offset to avoid overlap with points
    axes[0].annotate(name, (X0[i, 0], X0[i, 1]), 
                    xytext=(X0[i, 0] + 0.000001, X0[i, 1] + 0.000001))
axes[0].set_title('Dimension 0 (Connected Components)')

# Plot dimension 1 embedding (loops/cycles)
axes[1].scatter(X1[:, 0], X1[:, 1], c=range(l), cmap='viridis')
for i, name in enumerate(sample_names):
    # Add sample name labels with slight offset
    axes[1].annotate(name, (X1[i, 0], X1[i, 1]), 
                    xytext=(X1[i, 0] + 0.000001, X1[i, 1] + 0.000001))
axes[1].set_title('Dimension 1 (Loops)')

# Plot dimension 2 embedding (voids/cavities)
axes[2].scatter(X2[:, 0], X2[:, 1], c=range(l), cmap='viridis')
for i, name in enumerate(sample_names):
    # Add sample name labels with slight offset
    axes[2].annotate(name, (X2[i, 0], X2[i, 1]), 
                    xytext=(X2[i, 0] + 0.000001, X2[i, 1] + 0.000001))
axes[2].set_title('Dimension 2 (Voids)')

# Adjust layout and display the plot
plt.tight_layout()
plt.show()


# # Get sample names
# sample_names = [s[0] for s in samples]

# # Plot dimension 0 embedding
# plt.figure(figsize=(8, 6))
# plt.scatter(X0[:, 0], X0[:, 1], c=range(l), cmap='viridis')
# for i, name in enumerate(sample_names):
#     plt.annotate(name, (X0[i, 0], X0[i, 1]))
# plt.title('Dimension 0 (Connected Components)')
# plt.tight_layout()
# plt.show()

# # Plot dimension 1 embedding
# plt.figure(figsize=(8, 6))
# plt.scatter(X1[:, 0], X1[:, 1], c=range(l), cmap='viridis')
# for i, name in enumerate(sample_names):
#     plt.annotate(name, (X1[i, 0], X1[i, 1]))
# plt.title('Dimension 1 (Loops)')
# plt.tight_layout()
# plt.show()

# # Plot dimension 2 embedding
# plt.figure(figsize=(8, 6))
# plt.scatter(X2[:, 0], X2[:, 1], c=range(l), cmap='viridis')
# for i, name in enumerate(sample_names):
#     plt.annotate(name, (X2[i, 0], X2[i, 1]))
# plt.title('Dimension 2 (Voids)')
# plt.tight_layout()
# plt.show()

In [ ]:
# Create lists to store persistence intervals for each homological dimension
# pl0: connected components (dimension 0)
# pl1: loops/cycles (dimension 1) 
# pl2: voids/cavities (dimension 2)
pl0 = []
pl1 = []
pl2 = []

# Process each sample to extract persistence intervals
for name, points in samples:
    # Create Alpha Complex from point cloud
    ac = gd.DelaunayCechComplex(points=points)
    
    # Build simplex tree for topological analysis
    st = ac.create_simplex_tree()
    
    # Compute persistent homology
    bc = st.persistence()
    
    # Extract persistence intervals for each dimension and store them
    pl0.append(st.persistence_intervals_in_dimension(0))  # Connected components
    pl1.append(st.persistence_intervals_in_dimension(1))  # Loops/cycles
    pl2.append(st.persistence_intervals_in_dimension(2))  # Voids/cavities
    
    print(f"Persistence intervals for {name} appended.")

# # Display the shape of persistence intervals for each sample and dimension
# for i, name in enumerate([s[0] for s in samples]):
#     print(f"{name}: dim0={pl0[i].shape}, dim1={pl1[i].shape}, dim2={pl2[i].shape}")

# Initialize distance matrices for each homological dimension
# B0: bottleneck distances for connected components (dimension 0)
# B1: bottleneck distances for loops/cycles (dimension 1) 
# B2: bottleneck distances for voids/cavities (dimension 2)
B0 = np.zeros((l, l))
B1 = np.zeros((l, l))
B2 = np.zeros((l, l))

# Compute pairwise bottleneck distances between persistence intervals
# Only compute upper triangle to avoid redundant calculations
for i in range(l):
    for j in range(i):
        # Calculate bottleneck distance between persistence intervals of samples i and j
        B0[i,j] = gd.bottleneck_distance(pl0[i], pl0[j])  # Dimension 0
        B1[i,j] = gd.bottleneck_distance(pl1[i], pl1[j])  # Dimension 1
        B2[i,j] = gd.bottleneck_distance(pl2[i], pl2[j])  # Dimension 2
        print(f"Bottleneck distances for {samples[i][0]} and {samples[j][0]} computed.")

# Make distance matrices symmetric by adding the transpose
# This fills in the lower triangle with the same values as the upper triangle
B0 = B0 + B0.transpose()
B1 = B1 + B1.transpose()
B2 = B2 + B2.transpose()

# Apply MDS to each bottleneck distance matrix to get 2D coordinates
X0 = mds.fit_transform(B0)  # 2D embedding for dimension 0 (connected components)
X1 = mds.fit_transform(B1)  # 2D embedding for dimension 1 (loops/cycles)
X2 = mds.fit_transform(B2)  # 2D embedding for dimension 2 (voids/cavities)

# Create a single figure with 3 vertically stacked subplots
fig, axes = plt.subplots(3, 1, figsize=(10, 15))
sample_names = [s[0] for s in samples]  # Extract sample names for labeling

# Plot dimension 0 embedding (connected components)
axes[0].scatter(X0[:, 0], X0[:, 1], c=range(l), cmap='viridis')
for i, name in enumerate(sample_names):
    # Add sample name labels with slight offset to avoid overlap with points
    axes[0].annotate(name, (X0[i, 0], X0[i, 1]), 
                    xytext=(X0[i, 0] + 0.000001, X0[i, 1] + 0.000001))
axes[0].set_title('Dimension 0 (Connected Components)')

# Plot dimension 1 embedding (loops/cycles)
axes[1].scatter(X1[:, 0], X1[:, 1], c=range(l), cmap='viridis')
for i, name in enumerate(sample_names):
    # Add sample name labels with slight offset
    axes[1].annotate(name, (X1[i, 0], X1[i, 1]), 
                    xytext=(X1[i, 0] + 0.000001, X1[i, 1] + 0.000001))
axes[1].set_title('Dimension 1 (Loops)')

# Plot dimension 2 embedding (voids/cavities)
axes[2].scatter(X2[:, 0], X2[:, 1], c=range(l), cmap='viridis')
for i, name in enumerate(sample_names):
    # Add sample name labels with slight offset
    axes[2].annotate(name, (X2[i, 0], X2[i, 1]), 
                    xytext=(X2[i, 0] + 0.000001, X2[i, 1] + 0.000001))
axes[2].set_title('Dimension 2 (Voids)')

# Adjust layout and display the plot
plt.tight_layout()
plt.show()

In [2]:
# Create lists to store persistence intervals for each homological dimension
# pl0: connected components (dimension 0)
# pl1: loops/cycles (dimension 1) 
# pl2: voids/cavities (dimension 2)
pl0 = []
pl1 = []
pl2 = []

for name, points in samples:
    
    print(f"\nProcessing sample: {name}")

    # 1. Define Witness Points
    # The full set of data points will serve as witness points.
    witness_points = points

    # 2. Choose Landmark Points
    # We need to select a subset of `witness_points` to be `landmark_points`.
    # A common strategy is random sampling. The number of landmarks significantly affects the computational cost and the approximation quality of the complex.
    # For example, we choose a maximum of 500 landmarks, or 20% of total points, whichever is smaller, to keep the complex manageable.
    num_total_points = witness_points.shape[0]
    target_num_landmarks = min(500, int(0.2 * num_total_points))

    num_landmarks = max(1, target_num_landmarks)
    print(f"\nTotal points loaded: {num_total_points}")
    print(f"Number of landmark points chosen: {num_landmarks}")

    # Randomly select indices for landmark points without replacement
    landmark_indices = np.random.choice(num_total_points, num_landmarks, replace=False)
    landmark_points = witness_points[landmark_indices]

    # 3. Create the Euclidean Witness Complex
    # The `max_alpha_value` parameter acts as a maximum allowed "radius" for a simplex to be included in the complex.
    # - If too small, the complex might be empty or very sparse.
    # - If too large, it might be too dense and computationally expensive.
    # A heuristic to estimate `max_alpha_value` is to consider a fraction of the data's overall range or diameter.
    min_coords = np.min(witness_points, axis=0)
    max_coords = np.max(witness_points, axis=0)
    data_range = np.linalg.norm(max_coords - min_coords) # Diagonal length of bounding box
    
    # A common starting point: 5-15% of the data range
    estimated_max_alpha_value = data_range * 0.1
    
    # Fallback for very small or zero range (e.g., all points identical)
    if estimated_max_alpha_value < 1e-6:
        estimated_max_alpha_value = 1.0 

    max_alpha_value = estimated_max_alpha_value ** 2 # create_simplex_tree expects squared radius
    print(f"Estimated data range: {data_range:.8f}")
    print(f"Using max_alpha_value: {max_alpha_value:.8f}")
    
    witness_complex = gd.EuclideanWitnessComplex(
        landmarks=landmark_points,
        witnesses=witness_points
    )

    # 4. Create the Simplex Tree from the witness complex
    simplex_tree = witness_complex.create_simplex_tree(max_alpha_square=max_alpha_value, limit_dimension=2)

    print("\n--- Euclidean Witness Complex Information ---")
    print(f"Successfully created a Euclidean Witness Complex.")
    print(f"Number of simplices in the complex: {simplex_tree.num_simplices()}")
    print(f"Dimension of the complex: {simplex_tree.dimension()}")

    # 5. Compute persistent homology
    print("\nComputing persistent homology...")
    persistence = simplex_tree.persistence(persistence_dim_max=True)
    print(f"Number of persistence pairs: {len(persistence)}")

    # 6. Extract persistence intervals for each dimension and store them
    pl0.append(simplex_tree.persistence_intervals_in_dimension(0))  # Connected components
    pl1.append(simplex_tree.persistence_intervals_in_dimension(1))  # Loops/cycles
    pl2.append(simplex_tree.persistence_intervals_in_dimension(2))  # Voids/cavities

    print(f"Persistence intervals for {name} appended.")
            

# Initialize distance matrices for each homological dimension
# B0: bottleneck distances for connected components (dimension 0)
# B1: bottleneck distances for loops/cycles (dimension 1) 
# B2: bottleneck distances for voids/cavities (dimension 2)
B0 = np.zeros((l, l))
B1 = np.zeros((l, l))
B2 = np.zeros((l, l))

# Compute pairwise bottleneck distances between persistence intervals
# Only compute upper triangle to avoid redundant calculations
for i in range(l):
    for j in range(i):
        # Calculate bottleneck distance between persistence intervals of samples i and j
        B0[i,j] = gd.bottleneck_distance(pl0[i], pl0[j])  # Dimension 0
        B1[i,j] = gd.bottleneck_distance(pl1[i], pl1[j])  # Dimension 1
        B2[i,j] = gd.bottleneck_distance(pl2[i], pl2[j])  # Dimension 2
        print(f"Bottleneck distances for {samples[i][0]} and {samples[j][0]} computed.")

# Make distance matrices symmetric by adding the transpose
# This fills in the lower triangle with the same values as the upper triangle
B0 = B0 + B0.transpose()
B1 = B1 + B1.transpose()
B2 = B2 + B2.transpose()

# Apply MDS to each bottleneck distance matrix to get 2D coordinates
X0 = mds.fit_transform(B0)  # 2D embedding for dimension 0 (connected components)
X1 = mds.fit_transform(B1)  # 2D embedding for dimension 1 (loops/cycles)
X2 = mds.fit_transform(B2)  # 2D embedding for dimension 2 (voids/cavities)

# Create a single figure with 3 vertically stacked subplots
fig, axes = plt.subplots(3, 1, figsize=(10, 15))
sample_names = [s[0] for s in samples]  # Extract sample names for labeling

# Plot dimension 0 embedding (connected components)
axes[0].scatter(X0[:, 0], X0[:, 1], c=range(l), cmap='viridis')
for i, name in enumerate(sample_names):
    # Add sample name labels with slight offset to avoid overlap with points
    axes[0].annotate(name, (X0[i, 0], X0[i, 1]), 
                    xytext=(X0[i, 0] + 0.000001, X0[i, 1] + 0.000001))
axes[0].set_title('Dimension 0 (Connected Components)')

# Plot dimension 1 embedding (loops/cycles)
axes[1].scatter(X1[:, 0], X1[:, 1], c=range(l), cmap='viridis')
for i, name in enumerate(sample_names):
    # Add sample name labels with slight offset
    axes[1].annotate(name, (X1[i, 0], X1[i, 1]), 
                    xytext=(X1[i, 0] + 0.000001, X1[i, 1] + 0.000001))
axes[1].set_title('Dimension 1 (Loops)')

# Plot dimension 2 embedding (voids/cavities)
axes[2].scatter(X2[:, 0], X2[:, 1], c=range(l), cmap='viridis')
for i, name in enumerate(sample_names):
    # Add sample name labels with slight offset
    axes[2].annotate(name, (X2[i, 0], X2[i, 1]), 
                    xytext=(X2[i, 0] + 0.000001, X2[i, 1] + 0.000001))
axes[2].set_title('Dimension 2 (Voids)')

# Adjust layout and display the plot
plt.tight_layout()
plt.show()



Processing sample: 500
Total points loaded: 500
Number of landmark points chosen: 100
Estimated data range: 0.26182231
Using max_alpha_value: 0.00068551

--- Euclidean Witness Complex Information ---
Successfully created a Euclidean Witness Complex.
Number of simplices in the complex: 5950
Dimension of the complex: 2

Computing persistent homology...
Number of persistence pairs: 3818
Persistence intervals for 500 appended.

Processing sample: 750
Total points loaded: 750
Number of landmark points chosen: 150
Estimated data range: 0.26079845
Using max_alpha_value: 0.00068016

--- Euclidean Witness Complex Information ---
Successfully created a Euclidean Witness Complex.
Number of simplices in the complex: 16758
Dimension of the complex: 2

Computing persistent homology...
Number of persistence pairs: 12243
Persistence intervals for 750 appended.

Processing sample: 1000
Total points loaded: 1000
Number of landmark points chosen: 200
Estimated data range: 0.25888351
Using max_alpha_valu

ValueError: Input X contains infinity or a value too large for dtype('float64').

# Potential Issues with the Witness Complex Implementation

## Problem Analysis:

The main issues in the current Witness Complex code are:

### 1. **Missing Error Handling**
- The code removed the `try-except` block that was protecting against failures
- If the complex creation fails for any sample, the entire process crashes

### 2. **Incomplete Persistence Lists**
- If any sample fails to process, the persistence lists (pl0, pl1, pl2) will have different lengths
- This causes errors when computing bottleneck distances later

### 3. **Potential Parameter Issues**
- `max_alpha_value` might still be too restrictive for some datasets
- The witness complex might produce empty or very sparse results

### 4. **Missing Validation**
- No check if the complex actually contains meaningful simplices
- No validation that persistence intervals were successfully computed

## Solutions:

In [ ]:
# CORRECTED VERSION: Witness Complex with proper error handling and infinite distance handling
# Create lists to store persistence intervals for each homological dimension
pl0_witness = []
pl1_witness = []
pl2_witness = []
successful_samples = []  # Track which samples were successfully processed

for name, points in samples:
    print(f"\nProcessing sample: {name}")
    
    try:
        # 1. Define Witness Points
        witness_points = points
        
        # 2. Choose Landmark Points
        num_total_points = witness_points.shape[0]
        target_num_landmarks = min(500, int(0.2 * num_total_points))
        num_landmarks = max(1, target_num_landmarks)
        
        print(f"Total points: {num_total_points}, Landmarks: {num_landmarks}")
        
        # Randomly select landmark points
        landmark_indices = np.random.choice(num_total_points, num_landmarks, replace=False)
        landmark_points = witness_points[landmark_indices]
        
        # 3. Estimate max_alpha_value
        min_coords = np.min(witness_points, axis=0)
        max_coords = np.max(witness_points, axis=0)
        data_range = np.linalg.norm(max_coords - min_coords)
        
        # Try multiple alpha values if the first one fails
        alpha_multipliers = [0.1, 0.2, 0.3, 0.5]  # Different percentages of data range
        
        success = False
        for multiplier in alpha_multipliers:
            estimated_max_alpha_value = data_range * multiplier
            if estimated_max_alpha_value < 1e-6:
                estimated_max_alpha_value = 1.0
                
            max_alpha_value = estimated_max_alpha_value ** 2
            print(f"Trying max_alpha_value: {max_alpha_value:.8f} (multiplier: {multiplier})")
            
            try:
                # 4. Create Witness Complex
                witness_complex = gd.EuclideanWitnessComplex(
                    landmarks=landmark_points,
                    witnesses=witness_points
                )
                
                # 5. Create Simplex Tree
                simplex_tree = witness_complex.create_simplex_tree(
                    max_alpha_square=max_alpha_value, 
                    limit_dimension=2
                )
                
                # 6. Check if complex is meaningful
                num_simplices = simplex_tree.num_simplices()
                if num_simplices > 0:
                    print(f"Success! Complex has {num_simplices} simplices")
                    
                    # 7. Compute persistent homology
                    persistence = simplex_tree.persistence(persistence_dim_max=True)
                    
                    # 8. Extract persistence intervals
                    intervals_0 = simplex_tree.persistence_intervals_in_dimension(0)
                    intervals_1 = simplex_tree.persistence_intervals_in_dimension(1)
                    intervals_2 = simplex_tree.persistence_intervals_in_dimension(2)
                    
                    # Check if intervals are valid (not empty and finite)
                    valid_intervals = True
                    for intervals, dim_name in [(intervals_0, "Dim0"), (intervals_1, "Dim1"), (intervals_2, "Dim2")]:
                        if intervals.shape[0] == 0:
                            print(f"  Warning: {dim_name} has no intervals")
                        elif not np.all(np.isfinite(intervals)):
                            print(f"  Warning: {dim_name} contains infinite values")
                            valid_intervals = False
                    
                    if valid_intervals:
                        # Only add if we have valid intervals
                        pl0_witness.append(intervals_0)
                        pl1_witness.append(intervals_1)
                        pl2_witness.append(intervals_2)
                        successful_samples.append(name)
                        
                        print(f"✓ Sample {name} processed successfully")
                        print(f"  Intervals - Dim0: {intervals_0.shape}, Dim1: {intervals_1.shape}, Dim2: {intervals_2.shape}")
                        success = True
                        break
                    else:
                        print(f"Invalid intervals with multiplier {multiplier}")
                else:
                    print(f"Complex is empty with multiplier {multiplier}")
                    
            except Exception as inner_e:
                print(f"Failed with multiplier {multiplier}: {inner_e}")
                continue
        
        if not success:
            print(f"✗ Failed to process sample {name} with all alpha values")
            
    except Exception as e:
        print(f"✗ Error processing sample {name}: {e}")

print(f"\nSummary: Successfully processed {len(successful_samples)} out of {len(samples)} samples")
print(f"Successful samples: {successful_samples}")

# Only proceed if we have enough successful samples
if len(successful_samples) >= 2:
    print("Proceeding with analysis...")
    
    # Update the sample count for successful samples only
    l_witness = len(successful_samples)
    
    # Compute bottleneck distances only for successful samples
    B0_witness = np.zeros((l_witness, l_witness))
    B1_witness = np.zeros((l_witness, l_witness))
    B2_witness = np.zeros((l_witness, l_witness))
    
    print("Computing bottleneck distances...")
    for i in range(l_witness):
        for j in range(i):
            try:
                # Compute distances with error handling
                dist_0 = gd.bottleneck_distance(pl0_witness[i], pl0_witness[j])
                dist_1 = gd.bottleneck_distance(pl1_witness[i], pl1_witness[j])
                dist_2 = gd.bottleneck_distance(pl2_witness[i], pl2_witness[j])
                
                # Check for infinite distances and replace with a large finite value
                max_finite_distance = 1000.0  # Reasonable upper bound
                
                B0_witness[i,j] = min(dist_0, max_finite_distance) if np.isfinite(dist_0) else max_finite_distance
                B1_witness[i,j] = min(dist_1, max_finite_distance) if np.isfinite(dist_1) else max_finite_distance
                B2_witness[i,j] = min(dist_2, max_finite_distance) if np.isfinite(dist_2) else max_finite_distance
                
                print(f"Distances {successful_samples[i]}-{successful_samples[j]}: {B0_witness[i,j]:.4f}, {B1_witness[i,j]:.4f}, {B2_witness[i,j]:.4f}")
                
            except Exception as e:
                print(f"Error computing distance between {successful_samples[i]} and {successful_samples[j]}: {e}")
                # Use maximum distance as fallback
                B0_witness[i,j] = max_finite_distance
                B1_witness[i,j] = max_finite_distance
                B2_witness[i,j] = max_finite_distance
    
    # Make symmetric
    B0_witness = B0_witness + B0_witness.transpose()
    B1_witness = B1_witness + B1_witness.transpose()
    B2_witness = B2_witness + B2_witness.transpose()
    
    # Verify matrices are finite before MDS
    matrices = [(B0_witness, "B0"), (B1_witness, "B1"), (B2_witness, "B2")]
    for matrix, name in matrices:
        if not np.all(np.isfinite(matrix)):
            print(f"Warning: {name} contains non-finite values!")
            print(f"  Min: {np.min(matrix)}, Max: {np.max(matrix)}")
            print(f"  Infinite values: {np.sum(~np.isfinite(matrix))}")
    
    # Apply MDS with error handling
    print("Applying MDS...")
    try:
        X0_witness = mds.fit_transform(B0_witness)
        print("✓ MDS successful for dimension 0")
    except Exception as e:
        print(f"✗ MDS failed for dimension 0: {e}")
        X0_witness = np.random.randn(l_witness, 2)  # Fallback to random positions
    
    try:
        X1_witness = mds.fit_transform(B1_witness)
        print("✓ MDS successful for dimension 1")
    except Exception as e:
        print(f"✗ MDS failed for dimension 1: {e}")
        X1_witness = np.random.randn(l_witness, 2)  # Fallback to random positions
    
    try:
        X2_witness = mds.fit_transform(B2_witness)
        print("✓ MDS successful for dimension 2")
    except Exception as e:
        print(f"✗ MDS failed for dimension 2: {e}")
        X2_witness = np.random.randn(l_witness, 2)  # Fallback to random positions
    
    # Plot results
    fig, axes = plt.subplots(3, 1, figsize=(10, 15))
    
    axes[0].scatter(X0_witness[:, 0], X0_witness[:, 1], c=range(l_witness), cmap='viridis')
    for i, name in enumerate(successful_samples):
        axes[0].annotate(name, (X0_witness[i, 0], X0_witness[i, 1]), 
                        xytext=(5, 5), textcoords='offset points')
    axes[0].set_title('Witness Complex - Dimension 0 (Connected Components)')
    
    axes[1].scatter(X1_witness[:, 0], X1_witness[:, 1], c=range(l_witness), cmap='viridis')
    for i, name in enumerate(successful_samples):
        axes[1].annotate(name, (X1_witness[i, 0], X1_witness[i, 1]), 
                        xytext=(5, 5), textcoords='offset points')
    axes[1].set_title('Witness Complex - Dimension 1 (Loops)')
    
    axes[2].scatter(X2_witness[:, 0], X2_witness[:, 1], c=range(l_witness), cmap='viridis')
    for i, name in enumerate(successful_samples):
        axes[2].annotate(name, (X2_witness[i, 0], X2_witness[i, 1]), 
                        xytext=(5, 5), textcoords='offset points')
    axes[2].set_title('Witness Complex - Dimension 2 (Voids)')
    
    plt.tight_layout()
    plt.show()
    
else:
    print("Not enough successful samples to proceed with analysis.")


Processing sample: 500
Total points: 500, Landmarks: 100
Trying max_alpha_value: 0.00068551 (multiplier: 0.1)
Success! Complex has 5925 simplices
Invalid intervals with multiplier 0.1
Trying max_alpha_value: 0.00274204 (multiplier: 0.2)
Invalid intervals with multiplier 0.1
Trying max_alpha_value: 0.00274204 (multiplier: 0.2)
Success! Complex has 51690 simplices
Invalid intervals with multiplier 0.2
Trying max_alpha_value: 0.00616958 (multiplier: 0.3)
Success! Complex has 51690 simplices
Invalid intervals with multiplier 0.2
Trying max_alpha_value: 0.00616958 (multiplier: 0.3)
Success! Complex has 144364 simplices
Success! Complex has 144364 simplices
Invalid intervals with multiplier 0.3
Trying max_alpha_value: 0.01713773 (multiplier: 0.5)
Invalid intervals with multiplier 0.3
Trying max_alpha_value: 0.01713773 (multiplier: 0.5)
Success! Complex has 166750 simplices
Success! Complex has 166750 simplices
Invalid intervals with multiplier 0.5
✗ Failed to process sample 500 with all alp